# Parallelization engines

Even for large cases _foxes_ calculations are fast, thanks to

- **Vectorization:** The states (and also the points, in the case of data calculation at evaluation points) are split into so-called _chunks_, which are sub-arrays of the large original data.
- **Parallelization:** These chunks are being sent to individual processes for calculation. Those calculations can be carried out simultaneously, i.e., _in parallel_.  

Vectorization and parallelization are managed by so-called _engines_ in _foxes_. If you do not explicitly specify the engine, a default will be chosen. This means that even if you do not know or care about _foxes_ engines, your calculations will be vectorized and parallelized.

Let's first explore this default before moving on to the full range of options.

## Default engine

Let's start by importing _foxes_ and other required packages:

In [ ]:
import matplotlib.pyplot as plt

import foxes
import foxes.variables as FV

Next, we create a random wind farm and a random time series:

In [ ]:
n_times = 10000
n_turbines = 100
seed = 42

sdata = foxes.input.states.create.random_timseries_data(
    n_times, seed=seed,
)
states = foxes.input.states.Timeseries(
    data_source=sdata,
    output_vars=[FV.WS, FV.WD, FV.TI, FV.RHO],
    fixed_vars={FV.RHO: 1.225, FV.TI: 0.02},
)

farm = foxes.WindFarm()
foxes.input.farm_layout.add_random(
    farm,
    n_turbines,
    min_dist=500,
    turbine_models=["DTU10MW"],
    seed=seed,
    verbosity=0
)

In [ ]:
sdata

In [ ]:
foxes.output.FarmLayoutOutput(farm).get_figure(figsize=(6,6))
plt.show()

You can run the wind farm calculations by simply creating an algorithm and calling _farm\_calc_:

In [ ]:
algo = foxes.algorithms.Downwind(
    farm,
    states,
    rotor_model="centre",
    wake_models=["Bastankhah2014_linear_k004"],
    verbosity=1,
)

In [ ]:
farm_results = algo.calc_farm()
farm_results

During the very first calculation, the algorithm checks if an engine is already up and running. If not, the default engine is created. We can check the currently active engine by the following function:

In [ ]:
foxes.get_engine()

This shows that the default choice is the _MultiprocessEngine_. Note that the parameter choice _chunk\_size_states=None_ represents a default choice for the states chunking by the engine, and does not mean that there is no chunking in that dimension.

We can reset the engine by

In [ ]:
foxes.reset_engine()

such that no engine is active afterwards:

In [ ]:
print(foxes.get_engine(error=False, default=False))

## Available engines

These are the currently available engines:

| Short name    | Class name         | Base package | Description                        |
|---------------|--------------------|--------------|------------------------------------| 
| multiprocess  | MultiprocessEngine | [multiprocess](https://github.com/uqfoundation/multiprocess) |  Runs on a workstation/laptop, using multi-processing |
| dask          | DaskEngine         | [dask](https://www.dask.org/) | Runs on a workstation/laptop, using processes or threads |
| xarray        | XArrayEngine       | [xarray](https://docs.xarray.dev/en/stable/) | Runs on a workstation/laptop, involving [dask](https://www.dask.org/) through [apply_ufunc](https://docs.xarray.dev/en/stable/generated/xarray.apply_ufunc.html)|
| local_cluster | LocalClusterEngine | [distributed](https://distributed.dask.org/en/stable/) | Runs on a workstation/laptop, creates a virtual local cluster |
| slurm_cluster | SlurmClusterEngine | [dask_jobqueue](https://jobqueue.dask.org/en/latest/generated/dask_jobqueue.SLURMCluster.html) | Runs on a multi-node HPC cluster which is using SLURM |
| numpy         | NumpyEngine        | [numpy](https://numpy.org/) | Runs as a single chunk, without parallelization |

There are two ways how to select a non-default engine and set all its parameters, as we will explore in the following two sections. 

## Engine selection through the algorithm

If you are using one algorithm for all calculations, you can select the engine directly via the algorithm's constructor. Make sure the algorithm is created at the beginning of your script, in particular before creating images, since those might launch the default engine otherwise.

In [ ]:
algo = foxes.algorithms.Downwind(
    farm,
    states,
    rotor_model="centre",
    wake_models=["Bastankhah2014_linear_k004"],
    verbosity=0,
    engine="dask",
    n_procs=None,
    chunk_size_states=2000,
    chunk_size_points=4000,
)

Here the _DaskEngine_ class was selected, with default _n\_procs_ and a user choice of chunk sizes. Notice that the short name from the above table can be used instead of the full class name (which, however, would also work).

For the complete list of constructor arguments of each of the engine classes, please check the API section _foxes.engines_. Any argument of the engine constructor can directly be added to the constructor of the algorithm, and will then be passed on.

Let's re-run the calculation using the above selected engine:

In [ ]:
farm_results = algo.calc_farm()

We can always check the current engine, and reset it if desired:

In [ ]:
foxes.get_engine()

In [ ]:
foxes.reset_engine()
print(foxes.get_engine(error=False, default=None))

## Engine selection through a with-block

For longer, more involved scripts that for example create several images, or create several algorithm instances (for example in a loop), it is recommended to use a _with_ block for creating the engine.

This also increases the readability concerning the engine choice.

For cluster based engines, the _with_ block is always preferred over the algorithm based engine specification, since it ensure the propper shutdown of the cluster connection.

The syntax is straight forward. First, we re-create the algorithm, this time without engine specification:

In [ ]:
algo = foxes.algorithms.Downwind(
    farm,
    states,
    rotor_model="centre",
    wake_models=["Bastankhah2014_linear_k004"],
    verbosity=0,
)

Now we can do all the work within the engine _with_ block:

In [ ]:
with foxes.Engine.new(
    "local_cluster",
    n_procs=4,
    chunk_size_states=2000,
    chunk_size_points=10000
):
    farm_results = algo.calc_farm()
    
    o = foxes.output.FlowPlots2D(algo, farm_results)
    next(o.gen_states_fig_xy(FV.WS, resolution=30, figsize=(6, 6), states_isel=[0]))
    plt.show()

Notice the _Dashboard_ link which displays the progress and cluster load during the execution. 

After the computation the engine is not set, as expected:

In [ ]:
print(foxes.get_engine(error=False, default=False))